## 데이터 탐색

In [1]:
import pandas as pd 
import numpy as np

In [36]:
factory_df = pd.read_csv('../data/factory_glass_2016.csv', encoding= 'cp949')
factory_df.info() # 8145 data, 842 columns
factory_df.head()
len(factory_df.index)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8145 entries, 0 to 8144
Columns: 841 entries, dat...date.name. to R.Vac
dtypes: float64(834), int64(6), object(1)
memory usage: 52.3+ MB


8145

- 결측치 제거

In [37]:
# 각 컬럼의 결측치 수 확인
missing_values_columns = factory_df.isnull().sum() 
# print(missing_values_columns[missing_values_columns > 0])  # 결측치가 있는 컬럼만 출력
# print(missing_values_columns[missing_values_columns > 8000]) # 7개의 열 - 결측치 8145

drop_df = factory_df.drop(missing_values_columns[missing_values_columns == 8145].index, axis= 1) 
drop_df.info() # column 841 > 834 : 7개의 컬럼 삭제

missing_values_columns = drop_df.isnull().sum()
# print(missing_values_columns[missing_values_columns > 0]) # 127개의 column 결측치 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8145 entries, 0 to 8144
Columns: 834 entries, dat...date.name. to R.Vac
dtypes: float64(827), int64(6), object(1)
memory usage: 51.8+ MB


In [ ]:
# 각 인덱스의 결측치 수 확인
missing_values_rows = drop_df.isnull().sum(axis= 1)
print(missing_values_rows[missing_values_rows > 0]) # 3754 index : 1, 6310 index : 39, 8044 index : 87 -> 6310, 8044 index 삭제
print()
# 8044 -> 1은 보간법으로 채움 - 시계열 데이터의 경우 선형 보간법 혹은 시간 기반 보간법
drop_df = drop_df.drop(missing_values_rows[missing_values_rows > 0].index[-2:], axis= 0)
drop_df.reset_index(drop= True, inplace= True) # 8145 index > 8143 index
drop_df.info() # 
print()
# 행 삭제, 보간 적용 확인
missing_values_rows = drop_df.isnull().sum(axis= 1)
missing_values_rows[missing_values_rows > 0] # 없는 것 확인

3754    1
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8142 entries, 0 to 8141
Columns: 834 entries, dat...date.name. to R.Vac
dtypes: float64(827), int64(6), object(1)
memory usage: 51.8+ MB



Series([], dtype: int64)

In [ ]:
# 보간법 imputer 사용 방법
# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy = 'median')

In [70]:
# drop_df.info()
drop_df.select_dtypes(object) # 날짜
# 그 외 데이터는 모두 수치형 데이터

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# LEFT, RIGHT 이름의 공정 중 LEFT,RIGHT만 다르고 공정명 동일 - 중요 공정 파악 문제이므로 둘 중 하나만 남김
# 단 LEFT/RIGHT를 제외한 공정명 동일해야 하고 공정번호 다른건 OK
# UPPER, LOWER은 공정의 비가 동일하지 않아 다른 공정일 수 있어서 삭제하지 않음
# 결과 도출 시 LEFT가 중요하면 RIGHT도 중요한지 한 번 확인
features = drop_df.columns.tolist()
# for feature in features:
#     print(feature)

# LEFT, RIGHT / L.,R. / l_,r_ / _L, _R
lr_columns = drop_df.filter(regex=r'\b(LEFT|RIGHT|Left|Right|L\.|R\.|l_|r_|_L|_R)\b').columns.tolist()
print(len(lr_columns))
for column in lr_columns :
  print(column)

# drop_df.describe()


71
DB.N2.SCREEN.FLOW.LEFT.1FI49011_PV
DB.N2.SCREEN.FLOW.RIGHT.1FI49012_PV
Left.edge
Right.edge
TMP.TIN..BAY.1.LEFT.1TI30201.PV
TMP.TIN..BAY.1.RIGHT.1TI30202.PV
TMP.TIN..BAY.4.LEFT.1TI30203.PV
TMP.TIN..BAY.5.LEFT.1TI30205.PV
TMP.TIN..BAY.5.RIGHT.1TI30206.PV
TMP.TIN..BAY.7.LEFT.1TI30207.PV
TMP.TIN..BAY.7.RIGHT.1TI30208.PV
TMP.TIN..BAY.10.LEFT.1TI30209.PV
TMP.TIN..BAY.10.RIGHT.1TI30210.PV
X.10.BAY.LEFT.PRESSURE.1CLBAY10LEFT_CPV
X.10.BAY.RIGHT.PRESSURE.1CLBAY10RIGHT_CPV
X.1.BAY.LEFT.PRESSURE.1CLBAY1LEFT_CPV
X.1.BAY.RIGHT.PRESSURE.1CLBAY1RIGHT_CPV
X.7.BAY.LEFT.PRESSURE.1CLBAY7LEFT_CPV
X.7.BAY.RIGHT.PRESSURE.1CLBAY7RIGHT_CPV
RET.AMBIANT.LEFT.TMP.1TI42602.PV
RET.AMBIANT.RIGHT.TMP.1TI42603.PV
BATH.AMBIENT.3.BAY.RIGHT.1TI31002_PV
BATH.AMBIENT.3.BAY.LEFT.1TI31003_PV
DB.N2.SCREEN.FLOW.LEFT.1FI49011_PV.1
DB.N2.SCREEN.FLOW.RIGHT.1FI49012_PV.1
EXIT.LIP.PLATE.LEFT.1TI30604.PV
EXIT.LIP.PLATE.RIGHT.1TI30605.PV
E.L.DB.LEFT..N2.FLOW.1FI33501.PV
E.L.DB.RIGHT.N2.FLOW.1FI33502.PV
TMP.GLASS..LEFT.EXIT.1TIC30